In [148]:
from pymongo import MongoClient
import redis
from time import gmtime, strftime
import pandas as pd
import pprint
import numpy as np
from datetime import datetime, timedelta


In [219]:

client = MongoClient()
R = redis.StrictRedis(host='127.0.0.1', port=6379, db=0)
R.flushdb()
for k in R.scan_iter("stud:12:*"):
    R.delete(k)

In [150]:
##Creating DB 
db = client['stud12']
collection = db['Work']
collection.drop()



In [151]:
##add company function
#In this question, we'll look for the company name in the cache due to efficiency considerations.
#If found, it means the collection includes this value; otherwise, we'll add it  
#Redis can check the existence of keys only, not value!!

def add_company(company):
        if type(company) is dict:
            key='stud12:companies:' + company['company_name']                
            if R.exists(key):
                print('Company name exists in the collection')
            else:
                R.set(key,company['company_name'])
                collection.insert_one(company)
                print('Company name was added to the collection')
        else:
            print('You should insert a dictionary')


In [152]:
#the job ID should be unique for a given company  and automatically generated when a new job is added to the company     
def add_job(job,company):
    #check if company exists - Redis
    #check if position exists - Redis
        #if not add position s
        #count number of positions
    #update job id
    #update status
    key='stud12:companies:' + company
    key_position='stud12:'+company+':' +job['job_title']
    key_id='stud12:companies:id:' + company
    if R.exists(key): #if the company exsist in the collection we can continue
        if R.exists(key_position): #does the position exist?
            print('the following position:{} has been inserted already into the database'.format(job['job_title']))
        else:
            if not R.exists(key_id): #initialization of job id  
                R.set(key_id,0) 
            job_id=R.incr(key_id) #incrementation of job id
            R.set(key_id,job_id) 
            R.set(key_position,job_id)
            key_position_id='stud12:'+company+':' +str(job_id)
            R.set(key_position_id, job_id)
            key_status='stud12:status:' + company +':' + str(job_id)
            R.set(key_status, job['status'])
            job['JobId'] = R.get(key_id) #get job id value from Redis update in mongoDB
            collection.update_one({'company_name':company }, {'$push': {'positions': job}},upsert=True) 
            print('All set, the job was added')
    else:
        print('Company: {} does not exsist in the collection'.format(company))
        

In [153]:
def add_application(candidate, job_id,company_name):
#Check company is in collection
#check position is in collection in company
#check if position is open 
#check if candidate applied with unique e-mail
#all validations are through redis
#if not appliued add candidate
    # add timestamp to candidate application 
    key='stud12:companies:' + company_name 
    key_position_id='stud12:'+company_name+':' +str(job_id)
    key_status='stud12:status:' + company_name +':' + str(job_id)
    key_email='stud12:'+company_name +':' + str(job_id)+':'+ candidate['email']
    if R.exists(key): #if the company exsist in the collection we can continue
        if R.exists(key_position_id): #if position exists continue
            if R.get(key_status)=='open': # if position is open continue, 
                if not R.exists(key_email): #if e-mail does not exist, candidate will apply 
                    R.set(key_email, job_id)      
                    candidate['ApplicationDate'] = datetime.now()           
                    collection.update_one({'company_name':company_name,"positions.JobId":job_id}, {'$push': {'positions.$.Applications': candidate}}) 
                    print('Candidate added to applicants list')
            
                else: 
                    print('candidate already applied to job')
            else: 
                print('Position: {} is closed'.format(job_id))
        else: 
            print( 'Position: {} does not exist'.format(job_id)    
    else: 
        print('Company: {} does not exsist in the collection'.format(company_name))
        
        
        
        
    


SyntaxError: invalid syntax (<ipython-input-153-7df0bfda0a31>, line 28)

In [154]:
def update_job_status(company_name, job_id, new_status):
    #Check company is in collection
    #check position is in collection in company
    #check position's status
        #change status\leave as is
    key='stud12:companies:' + company_name 
    key_position_id='stud12:'+company_name+':' +str(job_id)
    key_status='stud12:status:' + company_name +':' + str(job_id)    
    if R.exists(key): #if the company exsist in the collection we can continue
        if R.exists(key_position_id): #if position exists continue
            if R.get(key_status)!=new_status: # check if status needs to be changed
                    R.set(key_status,new_status) # update in Redis
                    MyQuery= {"company_name":company_name, "positions.JobId":job_id}
                    #location="positions.{}".format(str(job_id))
                    MyValue= {"$set":{"positions.$.status":new_status,"positions.$.modifiedDate":strftime("%d-%m-%Y", gmtime())}}
                    collection.update_one(MyQuery,MyValue)
                    print('status is changed')
            else: 
                print('status is already {}'.format(new_status))
        else:  
            print( 'Position: {} does not exist'.format(job_id)) 
    else:
        print('Company: {} does not exsist in the collection'.format(company_name))



In [225]:
def show_number_of_jobs(location,job_title):
    #aggregate function according to:
    #position=open
    #location
    #job title
    
    agg_result= list(collection.aggregate([{"$unwind":'$positions'},{"$match":{'positions.job_title':job_title,'positions.status':'open','positions.location':location}},{'$group':{'_id':{},"num_jobs": {"$sum": 1}}}]))
    df=pd.DataFrame(agg_result)
    if df.empty:
        print('no open positions for location: {} and job: {}'.format(location,job_title))
    else:
        print df 
    

    

 

In [156]:
def show_candidates(company_name,job_id):
    aggregation_function = [{"$unwind":'$positions'},
                         {"$match":{"company_name":company_name,"positions.JobId":job_id}},{"$unwind":'$positions.Applications'},
                        {"$project":{"email":"$positions.Applications.email",
                                     "skills":"$positions.Applications.skills",
                                     "job_requirments":'$positions.requirements','_id':0}},
                                     {'$project':{'commonToBoth': 
                                      {"$size":[{ '$setIntersection': [ "$job_requirments", '$skills' ]}]}
                                                  ,'_id':'$email'}},
                        {"$sort":{"commonToBoth":1}}]
    pprint.pprint(list(collection.aggregate(aggregation_function)))

In [241]:
def count_jobs_by_date():
    if not list(collection.find({}))==[]:
        
    
        ##our assumtion in this implemtation: 1. new position inserted into the collection is open
         # changing status can be only once- from open to close only 


        # 1.Handeling open positions
        # Assuming publish date as an open position - counting the number of positions published per date     
        aggregation_function = collection.aggregate([
        {"$unwind":"$positions"},
        {"$group": {"_id": "$positions.publish_date", "open_positions": {"$sum": 1}}}])
        open_jobs=pd.DataFrame(list(aggregation_function))


        #2.Handeling closed positions
        #Assuming modified date as a close position(the change can be from open to close only)

        aggregation_function_close = collection.aggregate([
        {"$unwind":"$positions"},
        {"$match":{"positions.status":"close"}},
        {"$group": {"_id": "$positions.modifiedDate", "close_positions": {"$sum": 1}}}])

        ##Inserting the data into table 
        closed_jobs=pd.DataFrame(list(aggregation_function_close))


        ##merging tables -open and close jobs
        try:
            merged_df = pd.merge(open_jobs, closed_jobs, how="outer", on="_id")
        except: 
            merged_df=closed_jobs
            merged_df['open_positions']=pd.Series([0 for x in range(len(merged_df.index))])

        ##Sorting by date

        merged_df = merged_df.sort_values(by="_id")

        ##calculations
        merged_df["cumulative_open"]=merged_df['open_positions'].cumsum(skipna=True)
        merged_df["cumulative_close"]=merged_df['close_positions'].cumsum(skipna=True)
        merged_df=merged_df.replace(np.nan, 0)
        merged_df['final_open']=merged_df["cumulative_open"]-merged_df["cumulative_close"]

        ##Final data frames with the relevant columns 
        final_df=merged_df[["_id","final_open","cumulative_close"]]
        final=final_df.rename(columns={'_id': "Date", "final_open": "number_of_open_jobs","cumulative_close": "number_of_close_jobs"})
        #final=pd.to_datetime(final['Date'])

        ##Limiting results to 2020 
        start_date = datetime.strptime("01-01-2020",'%d-%m-%Y') 
        end_date = datetime.strptime("31-12-2020",'%d-%m-%Y') 

        final['Date'] = pd.to_datetime(final['Date'])  

        filtered_dates = final[(final['Date']>= start_date) & (final['Date']<=  end_date)]


        print filtered_dates 
    else:
        print ('collection is empty')





In [158]:
def count_candidates_by_job():

##Defining the duration of time of the retrieved results 
    N_DAYS_AGO = 30
    now = datetime.now() # current date and time
    n_days_ago = now - timedelta(days=N_DAYS_AGO)
   

    ##Unwinding candidates, filtering the data from the last 30 days and counting the numbers of applications
    pipeline = [
        {"$unwind": "$positions"},{"$match":{
       "positions.Applications.ApplicationDate":{"$gte":n_days_ago}}},
        {"$group": {'_id':{
            "ID": "$positions.JobId","company":"$company_name",
            "candidates": "$positions.Applications.email"}, "candidats": {"$sum":1 }}}
           
    ]

    df1=pd.DataFrame(list(collection.aggregate(pipeline)))
    
    ##creating new data frame for storing the relevant data
    col = ['job','candidates']
    df = pd.DataFrame(columns=col)

    for index, row in df1.iterrows():
        df.loc[index,'job']= row['_id']['ID']+ '_' +row['_id']['company']
        df.loc[index,'candidates']=len(row['_id']['candidates'])
    
    
    print df
   


update_job_status('TAU','2','close')

In [200]:
## run to see database model(You can find it in the attached word file as well)
list(collection.find({}))





[]

In [197]:
##recovery the rellevant keys we stored in redis:
def recovery():
    for doc in collection.find():
        company_name=doc['company_name']
        key = 'stud12:companies:' + company_name##key
        R.set(key,company_name)

        # recovery number of jobs per company 
        key_id='stud12:companies:id' + company_name
        if 'positions' in doc:
            value = len(doc['positions'])
            R.set(key_id,value)
         
        else:
            R.set(key_id,0)

        ## Recovery key position id,position name ans status
        if 'positions' in doc:
            for i in doc['positions']:
                print i
                key_position='stud12:'+company_name+':' +i['job_title']
                key_position_id='stud12:'+company_name+':' +str(i['JobId'])
                key_status='stud12:companies:status' + company_name + str(i['JobId'])
               
                ##Recovery email adress
                for email in i['Applications']:
                    email_address=email['email']

                    key_email='stud12:'+company_name +':' + str(i['JobId'])+':'+ email_address
                    R.set(key_email,i['JobId'])
                

                R.set(key_position,i['JobId'])
                R.set(key_position_id,i['JobId'])
                R.set(key_status,i['status'])
              
                        

In [227]:
def execution():
#operation 1
    add_company({'company_name':'TAU','company_description':'University'})
#operation 2
    add_job({'job_title':'bi developer', 'location': 'Tel Aviv',
'requirements':['python','big data','mongodb'],
'status':'open','publish_date':'01-02-2020'},'TAU')
#operation 3
    add_application({'candidate_name':'laura', 'email':'laura@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', 'skills': ['python','sql'],
'application_date':'01-02-2019 15:00:00'}, '1','TAU')
#operation 4
    update_job_status('TAU','1','close')
#operation 5
    show_number_of_jobs('Tel Aviv','bi developer')
#operation 6
    show_candidates('TAU','1')

#report1
    count_jobs_by_date()

#report2
    count_candidates_by_job()
    if not R.ping():
        recovery()



In [228]:
execution()

Company name exists in the collection
the following position:bi developer has been inserted already into the database
Position is closed
status is already close
no open positions for location: Tel Aviv and job: bi developer
[{u'_id': u'laura@gmail.com', u'commonToBoth': 1}]
        Date  number_of_open_jobs  number_of_close_jobs
0 2020-01-02                  1.0                   0.0
     job candidates
0  1_TAU          1
